# 2. Multiple-output model

This notebook provides an example of building a neural net with multiple outputs. Specifically, we are training two binary classifiers: one that classifies if a digit is odd or even, and the other classifies if a digital is smaller than 5.

---

We first load the MNIST data

In [1]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Using TensorFlow backend.


We preprocess the image as previously

In [2]:
from utils import preprocess

train_images = preprocess(train_images)
test_images = preprocess(test_images)

We need to make labels for two classifiers. Notice we don't need to one-hot encode the labels in this case since we are building two binary classifiers using sigmoid actication in the last layer


In [3]:
# for odd-or-even classifier
train_label_ooe = (train_labels % 2 == 0)
test_label_ooe = (test_labels % 2 == 0)

# for smaller-than-5 classifier
train_label_st5 = (train_labels < 5)
test_label_st5 = (test_labels < 5)

We can construct our neural networks now

In [4]:
from keras import layers
from keras import Input
from keras.models import Model

Construct the shared layers

In [5]:
inputs = Input(shape=(28, 28, 1))
x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = layers.MaxPool2D(2, 2)(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPool2D(2, 2)(x)

Construct two 'heads'

In [6]:
# odd-or-even head
ooe = layers.Flatten()(x)
ooe = layers.Dense(64, activation='relu')(ooe)
ooe = layers.Dense(1, activation='sigmoid')(ooe)

# smaller than 5 head
st5 = layers.Flatten()(x)
st5 = layers.Dense(64, activation='relu')(st5)
st5 = layers.Dense(1, activation='sigmoid')(st5)

Instantiate the model with 1 input and 2 outputs, and compile the model

In [7]:
model = Model(inputs, [ooe, st5])
model.compile(optimizer='rmsprop',
              loss=['binary_crossentropy', 'binary_crossentropy'], # a list of two losses
              metrics=['accuracy'])

Train model

In [8]:
model.fit(train_images, [train_label_ooe, train_label_st5], epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 62s - loss: 0.1935 - dense_2_loss: 0.0921 - dense_4_loss: 0.1013 - dense_2_acc: 0.9646 - dense_4_acc: 0.9609    
Epoch 2/5
60000/60000 [==============================] - 63s - loss: 0.0648 - dense_2_loss: 0.0305 - dense_4_loss: 0.0343 - dense_2_acc: 0.9895 - dense_4_acc: 0.9885    
Epoch 3/5
60000/60000 [==============================] - 62s - loss: 0.0448 - dense_2_loss: 0.0209 - dense_4_loss: 0.0238 - dense_2_acc: 0.9926 - dense_4_acc: 0.9915    
Epoch 4/5
60000/60000 [==============================] - 62s - loss: 0.0350 - dense_2_loss: 0.0162 - dense_4_loss: 0.0188 - dense_2_acc: 0.9946 - dense_4_acc: 0.9938    
Epoch 5/5
60000/60000 [==============================] - 64s - loss: 0.0287 - dense_2_loss: 0.0130 - dense_4_loss: 0.0157 - dense_2_acc: 0.9955 - dense_4_acc: 0.9950    


And test the model on unseen data

In [9]:
metrics = model.evaluate(test_images, [test_label_ooe, test_label_st5])

 9952/10000 [============================>.] - ETA: 0s

Note that we have 3 losses (two seperate and one combined) and 2 accuracies (the last two elements in the list)

In [10]:
metrics[-2:]

[0.99550000000000005, 0.9929]

The two classifiers did equally good!